In [16]:
#1.데이터 로드
import numpy as np
import pandas as pd
train = pd.read_csv('data/train.csv')

X_train = train.drop(columns='Survived')
y_train = train['Survived']

X_test = pd.read_csv('data/test.csv')
PassengerId = X_test['PassengerId'].copy()

In [17]:
#2.EDA
import seaborn as sns

def graph(variable):
    survived = train[train['Survived']==1][variable].value_counts()
    dead = train[train['Survived']==0][variable].value_counts()
    result = pd.DataFrame([survived,dead])
    result.index = ['Survived','Dead']
    result.T.plot(kind='bar', color=['skyblue','gray'], rot=0, figsize=(8,6))

#graph('Parch')

In [18]:
#3.데이터 정제
#print(X_train['Fare'].unique())
#print(X_train.select_dtypes('object').nunique())

#불필요한 컬럼 제거
X_train = X_train.drop(columns=['PassengerId','Name','Ticket','Cabin'])
X_test = X_test.drop(columns=['PassengerId','Name','Ticket','Cabin'])

#SibSp, Parch 컬럼 통합
X_train['Family'] = X_train['SibSp']+X_train['Parch']
X_train = X_train.drop(columns=['SibSp','Parch'])

X_test['Family'] = X_test['SibSp']+X_test['Parch']
X_test = X_test.drop(columns=['SibSp','Parch'])

In [19]:
#4.결측치 및 이상치 처리
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].mean())
X_test['Age'] = X_test['Age'].fillna(X_train['Age'].mean())

X_train['Embarked'] = X_train['Embarked'].fillna(X_train['Embarked'].value_counts().idxmax())
X_test['Embarked'] = X_test['Embarked'].fillna(X_train['Embarked'].value_counts().idxmax())

X_train['Fare'] = X_train['Fare'].fillna(X_train['Fare'].mean())
X_test['Fare'] = X_test['Fare'].fillna(X_train['Fare'].mean())

In [20]:
# #BK.이상치 처리

# # sns.boxplot(data=X_train,y='Fare')
# # print(X_train.sort_values(by='Fare',ascending=False))
# # print(X_test.sort_values(by='Fare',ascending=False))

# #Fare 특성과 다른 특성들과 상관분석 후 상관계수가 가장 높은 컬럼별 Fare 평균 값으로 이상치 대체
# Fare_By_Pclass_train = pd.DataFrame(X_train.groupby(['Pclass'])['Fare'].mean())
# X_train = pd.merge(X_train,Fare_By_Pclass_train,left_on='Pclass',right_on='Pclass',how='left')
# X_train['Fare'] = X_train['Fare_x']*(X_train['Fare_x']!=X_train['Fare_x'].max()) + X_train['Fare_y']*(X_train['Fare_x']==X_train['Fare_x'].max())
# X_train = X_train.drop(columns=['Fare_x','Fare_y'])

# Fare_By_Pclass_test = pd.DataFrame(X_test.groupby(['Pclass'])['Fare'].mean())
# X_test = pd.merge(X_test,Fare_By_Pclass_test,left_on='Pclass',right_on='Pclass',how='left')
# X_test['Fare'] = X_test['Fare_x']*(X_test['Fare_x']!=X_test['Fare_x'].max()) + X_test['Fare_y']*(X_test['Fare_x']==X_test['Fare_x'].max())
# X_test = X_test.drop(columns=['Fare_x','Fare_y'])

# #사분위수를 활용한 이상치 처리
# # Q1 = X_train['Fare'].quantile(0.25)
# # Q3 = X_train['Fare'].quantile(0.75)
# # IQR = Q3-Q1
# # X_train.loc[(X_train['Fare']<Q1-1.5*IQR)|(X_train['Fare']>Q3+1.5*IQR),'Fare']=X_train['Fare'].mean()

In [21]:
# #BK.기계학습을 활용한 이상치 처리
# from sklearn.ensemble import IsolationForest

# X_train_Sub = X_train[['Age','Family','Fare']]
# X_test_Sub = X_test[['Age','Family','Fare']]

# model = IsolationForest(contamination=0.1, random_state=999)
# model.fit(X_train_Sub)

# X_train['outliers'] = model.predict(X_train_Sub)
# X_test['outliers'] = model.predict(X_test_Sub)

# X_train.loc[X_train['outliers']==-1,'Fare']=X_train['Fare'].mean()
# X_train.loc[X_train['outliers']==-1,'Age']=X_train['Age'].mean()
# X_train.loc[X_train['outliers']==-1,'Family']=X_train['Family'].mean()

# X_test.loc[X_test['outliers']==-1,'Fare']=X_test['Fare'].mean()
# X_test.loc[X_test['outliers']==-1,'Age']=X_test['Age'].mean()
# X_test.loc[X_test['outliers']==-1,'Family']=X_test['Family'].mean()

# X_train = X_train.drop(columns='outliers')
# X_test = X_test.drop(columns='outliers')

In [22]:
# #BK.Age컬럼 결측치 대체법
# # 상관분석 실행
# # print(X_train[['Age','Fare','Pclass','Family']].corr()['Age'])

# # 상관계수가 높은 컬럼별로 group by해서 age 평균을 각 행의 결측치에 대체
# Age_By_train = pd.DataFrame(X_train.groupby(['Pclass','Family'])['Age'].mean())
# X_train = pd.merge(X_train, Age_By_train, left_on=['Pclass','Family'], right_on=['Pclass','Family'], how='left')
# X_train['Age'] = X_train.apply(lambda row: row['Age_y'] if np.isnan(row['Age_x']) else row['Age_x'], axis=1)
# X_train = X_train.drop(columns=['Age_x','Age_y'])

# Age_By_test = pd.DataFrame(X_test.groupby(['Pclass','Family'])['Age'].mean())
# X_test = pd.merge(X_test, Age_By_test, left_on=['Pclass','Family'], right_on=['Pclass','Family'], how='left')
# X_test['Age'] = X_test.apply(lambda row: row['Age_y'] if np.isnan(row['Age_x']) else row['Age_x'], axis=1)
# X_test = X_test.drop(columns=['Age_x','Age_y'])

In [23]:
# #BK.다중 대체법을 사용하여 결측치 대체
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer
# from sklearn.ensemble import RandomForestRegressor

# print(X_train[['Age','Family','Pclass','Fare']].corr()[['Age','Fare']])

# model = RandomForestRegressor(random_state=999)

# imputer = IterativeImputer(model, max_iter=10, random_state=999)

# imputer_fit_train = imputer.fit(X_train[['Age','Pclass','Family']])
# imputed_data_train = imputer_fit_train.transform(X_train[['Age','Pclass','Family']])
# X_train['Age'] = imputed_data_train[:,0]
# imputer_fit_train = imputer.fit(X_train[['Fare','Pclass']])
# imputed_data_train = imputer_fit_train.transform(X_train[['Fare','Pclass']])
# X_train['Fare'] = imputed_data_train[:,0]

# imputer_fit_test = imputer.fit(X_test[['Age','Pclass','Family']])
# imputed_data_test = imputer_fit_test.transform(X_test[['Age','Pclass','Family']])
# X_test['Age'] = imputed_data_test[:,0]
# imputer_fit_test = imputer.fit(X_test[['Fare','Pclass']])
# imputed_data_test = imputer_fit_test.transform(X_test[['Fare','Pclass']])
# X_test['Fare'] = imputed_data_test[:,0]

# # print(X_test.shape[0],X_test['Fare'].isna().sum())

# # test = pd.concat([pd.DataFrame(X_test['Fare']),pd.DataFrame(imputed_data_test[:,2])],axis=1)
# # print((test['Fare']==test[0]).astype(int).sum())

# # sns.countplot(data=X_test,x='Age')

# print(X_train.isna().sum())
# print(X_test.isna().sum())

In [24]:
#5.인코딩
X_train = pd.get_dummies(X_train,columns=['Sex','Embarked','Pclass'])
X_test = pd.get_dummies(X_test,columns=['Sex','Embarked','Pclass'])

X_train_enc = X_train.select_dtypes('bool').astype('int')
X_test_enc = X_test.select_dtypes('bool').astype('int')

In [25]:
#6.스케일링
from sklearn.preprocessing import StandardScaler

X_train_num = X_train.select_dtypes(exclude='bool').copy()
X_test_num = X_test.select_dtypes(exclude='bool').copy()

scale = StandardScaler().fit(X_train_num)

X_train_std = scale.transform(X_train_num)
X_test_std = scale.transform(X_test_num)

X_train_std = pd.DataFrame(X_train_std,columns=['Age','Family','Fare'])
X_test_std = pd.DataFrame(X_test_std,columns=['Age','Family','Fare'])

X_train = pd.concat([X_train_std,X_train_enc],axis=1)
X_test = pd.concat([X_test_std,X_test_enc],axis=1)

In [26]:
#7.특성 선택
#RFE
from sklearn.feature_selection import RFE, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

X = X_train.drop(columns=['Sex_male','Embarked_C','Pclass_1'])
y = y_train
test = X_test.drop(columns=['Sex_male','Embarked_C','Pclass_1'])

model = RandomForestClassifier(
                    max_depth=7,
                    n_estimators=200,
                    max_features='sqrt',
                    min_samples_split=2,
                    min_samples_leaf=1,
                    random_state=999)

rfe = RFE(model, n_features_to_select=6)  # 선택할 특성의 개수를 지정합니다.

all_columns = X.columns

# RFE 알고리즘을 사용하여 특성 선택을 수행합니다.
rfe_fit = rfe.fit(X, y)

# 선택된 특성들을 출력합니다.
selected_mask = rfe_fit.support_
print(all_columns[selected_mask])
print(rfe_fit.estimator_.feature_importances_)

X = X_train[['Age', 'Family', 'Fare', 'Sex_female', 'Embarked_S', 'Pclass_3']]
test = X_test[['Age', 'Family', 'Fare', 'Sex_female', 'Embarked_S', 'Pclass_3']]

Index(['Age', 'Family', 'Fare', 'Sex_female', 'Embarked_S', 'Pclass_3'], dtype='object')
[0.16663802 0.21722868 0.08560329 0.39942311 0.02429701 0.10680989]


In [27]:
# #BK.SelectKBest을 활용한 특성 선택 탐색
# from sklearn.feature_selection import SelectKBest, f_classif

# X = X_train.drop(columns=['Sex_male','Embarked_C','Pclass_1'])
# y = y_train
# test = X_test.drop(columns=['Sex_male','Embarked_C','Pclass_1'])

# all_columns = X.columns

# sel = SelectKBest(score_func=f_classif,k=6)
# sel_fit = sel.fit(X,y)
# X = sel_fit.transform(X)
# test = sel_fit.transform(test)

# selected_mask = sel_fit.get_support()
# print(all_columns[selected_mask])
# print(X)
# print(test)

# print(all_columns)
# print(sel_fit.scores_.astype('int'))

# X = X_train[['Age','Family','Sex_female','Embarked_S','Pclass_2','Pclass_3']]
# y = y_train
# test = X_test[['Age','Family','Sex_female','Embarked_S','Pclass_2','Pclass_3']]

In [28]:
# #BK.statsmodels.Logit() 통계적 분석 결과 표(p-value, coef 값)를 통한 특성 선택
# import statsmodels.api as sm

# X = X_train.drop(columns=['Sex_male','Embarked_C','Pclass_1'])
# y = y_train
# test = X_test.drop(columns=['Sex_male','Embarked_C','Pclass_1'])

# model = sm.Logit(y,X)
# model_fit = model.fit()
# print(model_fit.summary())

# X = X_train[['Age','Fare','Sex_female','Embarked_S','Pclass_2','Pclass_3']]
# y = y_train
# test = X_test[['Age','Fare','Sex_female','Embarked_S','Pclass_2','Pclass_3']]

In [29]:
#8.하이퍼파라미터 최적화
from skopt import BayesSearchCV

opt = BayesSearchCV(
        RandomForestClassifier(max_features='sqrt'),
        {
        'n_estimators': (10, 1000),   # 트리 개수
        'max_depth': (3, 20),         # 트리 최대 깊이
        'min_samples_split': (2, 10), # 분할을 위한 최소 샘플 수
        'min_samples_leaf': (1, 10)   # 잎 노드에 필요한 최소 샘플 수
        },
        n_iter=20,   # 반복 횟수
        cv=5,        # 교차 검증 폴드 수
        scoring='accuracy'
    )
opt.fit(X, y)
print(opt.best_params_)

OrderedDict([('max_depth', 11), ('min_samples_leaf', 4), ('min_samples_split', 2), ('n_estimators', 125)])


In [ ]:
# #BK.하이퍼파라미터 최적화(전체)
# from skopt import BayesSearchCV
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier

# # 모델 정의
# models = {
#     'LogisticRegression': LogisticRegression(),
#     'RandomForestClassifier': RandomForestClassifier(max_features='sqrt'),
#     'XGBClassifier': XGBClassifier(),
#     'LGBMClassifier': LGBMClassifier(verbose=-1)
#     # 'CatBoostClassifier': CatBoostClassifier()
# }

# # 하이퍼파라미터 탐색 범위 지정
# search_space = {
#     'LogisticRegression': {
#         'max_iter': (100, 1000),
#         'C': (0.01, 100)
#     },
#     'RandomForestClassifier': {
#         'n_estimators': (10, 1000),   # 트리 개수
#         'max_depth': (3, 20),         # 트리 최대 깊이
#         'min_samples_split': (2, 10), # 분할을 위한 최소 샘플 수
#         'min_samples_leaf': (1, 10)   # 잎 노드에 필요한 최소 샘플 수
#     },
#     'XGBClassifier': {
#         'n_estimators': (10, 1000),
#         'max_depth': (3, 20),
#         'min_child_weight': (1, 20),
#         'learning_rate': (0.001, 1.0),
#         'subsample': (0.1,1)
#     },
#     'LGBMClassifier': {
#         'n_estimators': (10, 1000),
#         'max_depth': (3, 20),
#         'min_child_weight': (1, 20),
#         'learning_rate': (0.001, 1.0),
#         'num_leaves': (10,1000)
#     }
#     # 'CatBoostClassifier': {
#     #     'iterations': (10, 1000),
#     #     'depth': (3, 16),
#     #     'learning_rate': (0.001, 1.0)
#     # }
# }

# # 베이지안 최적화를 사용한 자동화된 하이퍼파라미터 튜닝
# best_params = {}
# for model_name, model in models.items():
#     opt = BayesSearchCV(
#         model,
#         search_space[model_name],
#         n_iter=10,   # 반복 횟수
#         cv=5,        # 교차 검증 폴드 수
#         scoring='accuracy'
#     )
#     opt.fit(X, y)
#     best_params[model_name] = opt.best_params_

# # 최적 하이퍼파라미터 출력
# for model_name, params in best_params.items():
#     print(model_name, "best parameter:", params)

In [31]:
#9.모델 생성

#LogisticRegression
model_lr = LogisticRegression(
                    max_iter=867,
                    C=49.475,
                    random_state=999)
# model_lr.fit(X,y)
# y_predict_lr = model_lr.predict(test)

#RandomForestClassifier
model_rf = RandomForestClassifier(
                    max_features='sqrt',
                    max_depth=7,
                    n_estimators=200,
                    min_samples_split=2,
                    min_samples_leaf=1,
                    random_state=999)
# model_rf.fit(X,y)
# y_predict_rf = model_rf.predict(test)

#XGBClassifier
model_xgb = XGBClassifier(
                    max_depth=14,
                    n_estimators=720,
                    min_child_weight=17,
                    learning_rate=0.264,
                    subsample=0.971,
                    random_state=999)
# model_xgb.fit(X,y)
# y_predict_xgb = model_xgb.predict(test)

#LGBMClassifier
model_lgbm = LGBMClassifier(
                    max_depth=19,
                    n_estimators=36,
                    min_child_weight=11,
                    learning_rate=0.972,
                    num_leaves=287,
                    verbose=-1,
                    random_state=999)
# model_lgbm.fit(X,y)
# y_predict_lgbm = model_lgbm.predict(test)

#CatBoostClassifier
# model_cat = CatBoostClassifier(
#                     depth=2,
#                     iterations=100,
#                     learning_rate=0.2)
# model_cat.fit(X,y,verbose=False)
# y_predict_cat = np.array(model_cat.predict(test)).flatten() #array in array 형태를 array로 변환

In [32]:
#10.모델 학습 및 평가

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score

kfold = KFold(n_splits=5,shuffle=True,random_state=999)
print(cross_validate(model_lr, X, y, cv=kfold, scoring='accuracy')['test_score'].mean().round(3))
print(cross_validate(model_rf, X, y, cv=kfold, scoring='accuracy')['test_score'].mean().round(3))
print(cross_validate(model_xgb, X, y, cv=kfold, scoring='accuracy')['test_score'].mean().round(3))
print(cross_validate(model_lgbm, X, y, cv=kfold, scoring='accuracy')['test_score'].mean().round(3))
# print(cross_validate(model_cat, X, y, cv=kfold, scoring='accuracy',fit_params={"verbose":False})['test_score'].mean().round(3))

0.808
0.825
0.822
0.828


In [33]:
# #BK.홀드아웃 기법
# from sklearn.model_selection import train_test_split
# X1, X2, y1, y2 = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=999)

# print(accuracy_score(y2,y_predict_lr).round(3))
# print(f1_score(y2,y_predict_lr).round(3))
# probas_lr = model_lr.predict_proba(X2)
# print(roc_auc_score(y2,probas_lr[:,1]).round(3))

In [34]:
#11.실 데이터 예측 및 제출
model_rf = RandomForestClassifier(
                    max_depth=7,
                    n_estimators=200,
                    max_features='sqrt',
                    min_samples_split=2,
                    min_samples_leaf=1,
                    random_state=999)
model_rf.fit(X,y)
y_predict_rf = model_rf.predict(test)

'''
#Best Model
model_rf = RandomForestClassifier(
                    max_depth=7,
                    n_estimators=200,
                    max_features='sqrt',
                    min_samples_split=2,
                    min_samples_leaf=1,
                    random_state=999)
model_rf.fit(X,y)
y_predict_rf = model_rf.predict(test)
'''

obj = {'PassengerId':PassengerId,'Survived':y_predict_rf}
result = pd.DataFrame(obj)
result.to_csv('result/result.csv', index = False)